In [24]:
import random

import numpy as np
import pandas as pd

from Uno.Game import Game
from Players.WDSRVCW4_Player import WDSRVCW4_Player
from Players.WDSRCVW4_Player import WDSRCVW4_Player
from Players.DSRVCWW4_Player import DSRVCWW4_Player
from Players.DSRCVWW4_Player import DSRCVWW4_Player
from Players.SRCVWDW4_Player import SRCVWDW4_Player
from Players.SRVCWDW4_Player import SRVCWDW4_Player

## Manually Simulate a Game
(for your fun/leisure)  
this code is also used to debug game rules

In [25]:
# set up the players
players = [WDSRVCW4_Player(), WDSRVCW4_Player(), WDSRVCW4_Player(), WDSRVCW4_Player()]

# create a game (with 381 as the random seed)
#   change the random seed to get a new game

#   when the seed is 787 and all 4 players are WDSRVCW4_Player
#       the game lasts a long time
game = Game(players, 381)
print(game)


Deck: [BLUE N5,GREEN N9,YELLOW N3,RED N8,YELLOW N1]
Discard: []

Turn: 0
0:	7 [YELLOW N6,YELLOW N5,YELLOW DRAW2,RED N2,GREEN N0,RED N7,GREEN N6]
1:	7 [BLUE N4,GREEN N1,YELLOW N9,BLUE N2,BLUE N0,BLUE N2,BLUE N1]
2:	7 [GREEN N5,BLUE N3,GREEN N2,GREEN DRAW2,BLUE N4,GREEN N2,YELLOW N5]
3:	7 [BLUE DRAW2,RED N1,RED N1,BLUE SKIP,YELLOW N2,RED REV,BLUE REV]
        


In [26]:
# run the setup step
game.setup_round()
print(game)


Deck: [GREEN N9,YELLOW N3,RED N8,YELLOW N1,YELLOW N4]
Discard: [BLUE N5]

Turn: 0
0:	7 [YELLOW N6,YELLOW N5,YELLOW DRAW2,RED N2,GREEN N0,RED N7,GREEN N6]
1:	7 [BLUE N4,GREEN N1,YELLOW N9,BLUE N2,BLUE N0,BLUE N2,BLUE N1]
2:	7 [GREEN N5,BLUE N3,GREEN N2,GREEN DRAW2,BLUE N4,GREEN N2,YELLOW N5]
3:	7 [BLUE DRAW2,RED N1,RED N1,BLUE SKIP,YELLOW N2,RED REV,BLUE REV]
        


In [28]:
# to play more rounds:
#  please re-run this cell until a player runs out of cards

# play rounds until a winner emerges
winner = game.round()
print(game)
if winner != -1:
    print('WINNER: ', winner)


Deck: [GREEN N9,YELLOW N3,RED N8,YELLOW N1,YELLOW N4]
Discard: [YELLOW N9,YELLOW N5,BLUE N5]

Turn: 2
0:	6 [YELLOW DRAW2,YELLOW N6,RED N2,GREEN N0,RED N7,GREEN N6]
1:	6 [BLUE N4,GREEN N1,BLUE N2,BLUE N0,BLUE N2,BLUE N1]
2:	7 [GREEN N5,BLUE N3,GREEN N2,GREEN DRAW2,BLUE N4,GREEN N2,YELLOW N5]
3:	7 [BLUE DRAW2,RED N1,RED N1,BLUE SKIP,YELLOW N2,RED REV,BLUE REV]
        


# Sampling

In [5]:
N_SAMPLES = 10000
N_EXPERIMENTS = 10

In [6]:
def play_game(your_strategy, opponent_strategy, n_players=4):
    # set up the players
    players = [None]*n_players
    players[0] = your_strategy()
    for i in range(1, n_players):
        players[i] = opponent_strategy()
    # create a game
    #   no seed for maximum randomness
    game = Game(players, None)
    game.setup_round()
    # play rounds until a winner emerges
    winner = game.round()
    while winner == -1:
        winner = game.round()
    return winner

In [7]:
def win_percent(your_strategy, opponent_strategy, n_players=4, n_samples=N_SAMPLES):
    your_wins = 0
    for _ in range(n_samples):
        if play_game(your_strategy, opponent_strategy, n_players) == 0:
            your_wins += 1
    return your_wins/n_samples

In [8]:
def win_percent_ci(your_strategy, opponent_strategy, n_players=4, n_samples=N_SAMPLES, n_exprs=N_EXPERIMENTS):
    expr_percents = np.zeros(n_exprs)

    for i in range(n_exprs):
        expr_percents[i] = win_percent(your_strategy, opponent_strategy, n_players, n_samples)

    mean_percent = np.mean(expr_percents)
    moe = 2**(-n_exprs+1)
    return (mean_percent-moe, mean_percent+moe)

In [ ]:
# round, str concat two float columns as interval
def make_CI_str_col(r, lo_col='low_prct', hi_col='high_prct'):
    r['CI'] = f'({round(r[lo_col], 3):.3f}, {round(r[hi_col], 3):.3f})'
    return r

In [9]:
from Players.WDSRVCW4_Player import WDSRVCW4_Player
from Players.WDSRCVW4_Player import WDSRCVW4_Player
from Players.DSRVCWW4_Player import DSRVCWW4_Player
from Players.DSRCVWW4_Player import DSRCVWW4_Player
from Players.SRCVWDW4_Player import SRCVWDW4_Player
from Players.SRVCWDW4_Player import SRVCWDW4_Player

In [11]:
strategy_names = ['WDSRVCW4', 'WDSRCVW4', 'DSRVCWW4', 'DSRCVWW4', 'SRCVWDW4', 'SRVCWDW4']
my_strat_list, opp_strat_list = [], []
low_perct_list, high_perct_list = [], []

# analysis results in './Results/basic_results.csv'
#   analysis took 65m 10sec
if False:
    simulate_games(
        (WDSRVCW4_Player, WDSRCVW4_Player, DSRVCWW4_Player,
         DSRCVWW4_Player, SRCVWDW4_Player, SRVCWDW4_Player),
        ['WDSRVCW4', 'WDSRCVW4', 'DSRVCWW4', 'DSRCVWW4', 'SRCVWDW4', 'SRVCWDW4'],
        'Results/basic_results.csv'
    )

In [ ]:
def simulate_games(player_list, strategy_names, out_file):
    for i, my_player in enumerate(player_list):
        for j, opp_player in enumerate(player_list):
                # save names of strategies at play
                my_strat_list.append(strategy_names[i])
                opp_strat_list.append(strategy_names[j])
                # simulate 10 runs of 10K games and get a CI
                low_perct, high_prct = win_percent_ci(my_player, opp_player)
                # save CI
                low_perct_list.append(low_perct)
                high_perct_list.append(high_prct)

        results_df = pd.DataFrame({
        'my_strat':my_strat_list,
        'opp_strat':opp_strat_list,
        'low_prct':low_perct_list,
        'high_prct':high_perct_list
        })
    # make a 'str' type column for the CI
    results_df = results_df.apply(make_CI_str_col, axis=1)
    # pivot the DF so we have 1 col/opponent strategy, 1 row/our strategy, 1 cell/CI
    basic_results_df = results_df.pivot(index='my_strat', columns='opp_strat', values='CI')
    if False:
        # save results
        basic_results_df.to_csv(out_file)